In [0]:
# Vérification de l'environnement (compatible serverless)
print("=" * 50)
print("ENVIRONNEMENT DATABRICKS")
print("=" * 50)
print(f"Version Spark : {spark.version}")
import sys
print(f"Langage : Python {sys.version.split()[0]}")
print(f"Type de cluster : Serverless")
print(f"Prêt pour la formation !")
print("=" * 50)

ENVIRONNEMENT DATABRICKS
✅ Version Spark : 4.0.0
✅ Langage : Python 3.12.3
✅ Type de cluster : Serverless
✅ Prêt pour la formation !


In [0]:
# Créer un dataset d'exemple : ventes d'un e-commerce
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from datetime import datetime

# Données brutes (simulation de fichier CSV)
data_ventes = [
    (1, "2025-10-01 10:30:00", "Alice", "Laptop", 1200.00, "Paris", "Confirmée"),
    (2, "2025-10-01 11:15:00", "Bob", "Souris", 25.50, "Lyon", "Confirmée"),
    (3, "2025-10-02 09:20:00", "Claire", "Clavier", 75.00, "Paris", "Annulée"),
    (4, "2025-10-02 14:45:00", "David", "Écran", 350.00, "Marseille", "Confirmée"),
    (5, "2025-10-03 16:00:00", "Emma", "Laptop", 1200.00, "Lyon", "Confirmée"),
    (6, "2025-10-03 17:30:00", "Alice", "Souris", 25.50, "Paris", "Confirmée"),
    (7, "2025-10-04 10:00:00", None, "Clavier", 75.00, None, "En attente"),
    (8, "2025-10-04 12:00:00", "Bob", "Écran", -50.00, "Lyon", "Confirmée"),
]

# Schéma du DataFrame
schema = StructType([
    StructField("id_commande", IntegerType(), True),
    StructField("date_commande", StringType(), True),
    StructField("client", StringType(), True),
    StructField("produit", StringType(), True),
    StructField("montant", DoubleType(), True),
    StructField("ville", StringType(), True),
    StructField("statut", StringType(), True)
])

# Créer le DataFrame (couche BRONZE - données brutes)
df_bronze = spark.createDataFrame(data_ventes, schema)

print("COUCHE BRONZE - Données brutes chargées")
print(f"Nombre de lignes : {df_bronze.count()}")
df_bronze.show()

COUCHE BRONZE - Données brutes chargées
Nombre de lignes : 8
+-----------+-------------------+------+-------+-------+---------+----------+
|id_commande|      date_commande|client|produit|montant|    ville|    statut|
+-----------+-------------------+------+-------+-------+---------+----------+
|          1|2025-10-01 10:30:00| Alice| Laptop| 1200.0|    Paris| Confirmée|
|          2|2025-10-01 11:15:00|   Bob| Souris|   25.5|     Lyon| Confirmée|
|          3|2025-10-02 09:20:00|Claire|Clavier|   75.0|    Paris|   Annulée|
|          4|2025-10-02 14:45:00| David|  Écran|  350.0|Marseille| Confirmée|
|          5|2025-10-03 16:00:00|  Emma| Laptop| 1200.0|     Lyon| Confirmée|
|          6|2025-10-03 17:30:00| Alice| Souris|   25.5|    Paris| Confirmée|
|          7|2025-10-04 10:00:00|  NULL|Clavier|   75.0|     NULL|En attente|
|          8|2025-10-04 12:00:00|   Bob|  Écran|  -50.0|     Lyon| Confirmée|
+-----------+-------------------+------+-------+-------+---------+----------+



In [0]:
from pyspark.sql.functions import col, to_timestamp, when, trim

print("TRANSFORMATION : BRONZE → SILVER")
print("=" * 50)

# 1. Convertir la date en format Timestamp
df_silver = df_bronze.withColumn(
    "date_commande", 
    to_timestamp(col("date_commande"), "yyyy-MM-dd HH:mm:ss")
)

# 2. Filtrer les lignes avec des valeurs NULL dans les colonnes critiques
df_silver = df_silver.filter(
    col("client").isNotNull() & 
    col("ville").isNotNull()
)

# 3. Filtrer les montants invalides (négatifs ou nuls)
df_silver = df_silver.filter(
    (col("montant") > 0)
)

# 4. Nettoyer les espaces dans les textes
df_silver = df_silver.withColumn("client", trim(col("client")))
df_silver = df_silver.withColumn("ville", trim(col("ville")))
df_silver = df_silver.withColumn("produit", trim(col("produit")))

# 5. Ajouter une colonne calculée : catégorie de prix
df_silver = df_silver.withColumn(
    "categorie_prix",
    when(col("montant") < 50, "Économique")
    .when((col("montant") >= 50) & (col("montant") < 500), "Standard")
    .otherwise("Premium")
)

print(f"Lignes avant nettoyage : {df_bronze.count()}")
print(f"Lignes après nettoyage : {df_silver.count()}")
print(f"Lignes supprimées : {df_bronze.count() - df_silver.count()}")
print("\n COUCHE SILVER - Données nettoyées :")
df_silver.show()

TRANSFORMATION : BRONZE → SILVER
Lignes avant nettoyage : 8
Lignes après nettoyage : 6
Lignes supprimées : 2

 COUCHE SILVER - Données nettoyées :
+-----------+-------------------+------+-------+-------+---------+---------+--------------+
|id_commande|      date_commande|client|produit|montant|    ville|   statut|categorie_prix|
+-----------+-------------------+------+-------+-------+---------+---------+--------------+
|          1|2025-10-01 10:30:00| Alice| Laptop| 1200.0|    Paris|Confirmée|       Premium|
|          2|2025-10-01 11:15:00|   Bob| Souris|   25.5|     Lyon|Confirmée|    Économique|
|          3|2025-10-02 09:20:00|Claire|Clavier|   75.0|    Paris|  Annulée|      Standard|
|          4|2025-10-02 14:45:00| David|  Écran|  350.0|Marseille|Confirmée|      Standard|
|          5|2025-10-03 16:00:00|  Emma| Laptop| 1200.0|     Lyon|Confirmée|       Premium|
|          6|2025-10-03 17:30:00| Alice| Souris|   25.5|    Paris|Confirmée|    Économique|
+-----------+------------

In [0]:
from pyspark.sql.functions import sum, count, avg, max, min

print("AGRÉGATION : SILVER → GOLD")
print("=" * 50)

# Métriques par ville
df_gold_ville = df_silver.groupBy("ville").agg(
    count("id_commande").alias("nombre_commandes"),
    sum("montant").alias("chiffre_affaires"),
    avg("montant").alias("panier_moyen"),
    count("client").alias("nombre_clients")
).orderBy(col("chiffre_affaires").desc())

print("\n COUCHE GOLD - Métriques par ville :")
df_gold_ville.show()

# Métriques par produit
df_gold_produit = df_silver.groupBy("produit").agg(
    count("id_commande").alias("quantite_vendue"),
    sum("montant").alias("revenu_total"),
    avg("montant").alias("prix_moyen")
).orderBy(col("revenu_total").desc())

print("\n COUCHE GOLD - Métriques par produit :")
df_gold_produit.show()

# Vue d'ensemble
print("\n RÉSUMÉ GLOBAL :")
print(f"Chiffre d'affaires total : {df_silver.agg(sum('montant')).collect()[0][0]:.2f} €")
print(f"Nombre total de commandes : {df_silver.count()}")
print(f"Nombre de villes : {df_silver.select('ville').distinct().count()}")
print(f"Nombre de produits : {df_silver.select('produit').distinct().count()}")

AGRÉGATION : SILVER → GOLD

 COUCHE GOLD - Métriques par ville :
+---------+----------------+----------------+------------+--------------+
|    ville|nombre_commandes|chiffre_affaires|panier_moyen|nombre_clients|
+---------+----------------+----------------+------------+--------------+
|    Paris|               3|          1300.5|       433.5|             3|
|     Lyon|               2|          1225.5|      612.75|             2|
|Marseille|               1|           350.0|       350.0|             1|
+---------+----------------+----------------+------------+--------------+


 COUCHE GOLD - Métriques par produit :
+-------+---------------+------------+----------+
|produit|quantite_vendue|revenu_total|prix_moyen|
+-------+---------------+------------+----------+
| Laptop|              2|      2400.0|    1200.0|
|  Écran|              1|       350.0|     350.0|
|Clavier|              1|        75.0|      75.0|
| Souris|              2|        51.0|      25.5|
+-------+---------------+-

In [0]:
print("SAUVEGARDE DANS DELTA LAKE")
print("=" * 50)

# Définir le chemin de stockage (DBFS = Databricks File System)
path_bronze = "/tmp/formation/bronze/ventes"
path_silver = "/tmp/formation/silver/ventes"
path_gold_ville = "/tmp/formation/gold/ventes_par_ville"
path_gold_produit = "/tmp/formation/gold/ventes_par_produit"

# Sauvegarder Bronze
df_bronze.write.format("delta").mode("overwrite").save(path_bronze)
print(f"Couche BRONZE sauvegardée : {path_bronze}")

# Sauvegarder Silver
df_silver.write.format("delta").mode("overwrite").save(path_silver)
print(f"Couche SILVER sauvegardée : {path_silver}")

# Sauvegarder Gold
df_gold_ville.write.format("delta").mode("overwrite").save(path_gold_ville)
print(f"Couche GOLD (ville) sauvegardée : {path_gold_ville}")

df_gold_produit.write.format("delta").mode("overwrite").save(path_gold_produit)
print(f"Couche GOLD (produit) sauvegardée : {path_gold_produit}")

print("\n Pipeline terminée avec succès !")

SAUVEGARDE DANS DELTA LAKE
Couche BRONZE sauvegardée : /tmp/formation/bronze/ventes
Couche SILVER sauvegardée : /tmp/formation/silver/ventes
Couche GOLD (ville) sauvegardée : /tmp/formation/gold/ventes_par_ville
Couche GOLD (produit) sauvegardée : /tmp/formation/gold/ventes_par_produit

 Pipeline terminée avec succès !


In [0]:
print("CRÉATION DES TABLES MANAGÉES")
print("=" * 70)

# Créer le schema
spark.sql("CREATE SCHEMA IF NOT EXISTS formation")
print("Schema 'formation' créé")

# Vérifier que les DataFrames existent
print("\n🔍 Vérification des DataFrames...")
try:
    print(f"  df_bronze : {df_bronze.count()} lignes")
    print(f"  df_silver : {df_silver.count()} lignes")
    print(f"  df_gold_ville : {df_gold_ville.count()} lignes")
    print(f"  df_gold_produit : {df_gold_produit.count()} lignes")
except:
    print("Les DataFrames n'existent pas. Exécutez d'abord les cellules 2, 3 et 4 !")
    raise

print("\nCréation des tables...")

# Bronze
df_bronze.write.format("delta").mode("overwrite").saveAsTable("formation.ventes_bronze")
print("Table formation.ventes_bronze créée")

# Silver
df_silver.write.format("delta").mode("overwrite").saveAsTable("formation.ventes_silver")
print("Table formation.ventes_silver créée")

# Gold - Ville
df_gold_ville.write.format("delta").mode("overwrite").saveAsTable("formation.ventes_gold_ville")
print("Table formation.ventes_gold_ville créée")

# Gold - Produit
df_gold_produit.write.format("delta").mode("overwrite").saveAsTable("formation.ventes_gold_produit")
print("Table formation.ventes_gold_produit créée")

print("\n Toutes les tables sont prêtes !")
print("Les données sont gérées automatiquement par Databricks")

CRÉATION DES TABLES MANAGÉES
Schema 'formation' créé

🔍 Vérification des DataFrames...
  df_bronze : 8 lignes
  df_silver : 6 lignes
  df_gold_ville : 3 lignes
  df_gold_produit : 4 lignes

Création des tables...
Table formation.ventes_bronze créée
Table formation.ventes_silver créée
Table formation.ventes_gold_ville créée
Table formation.ventes_gold_produit créée

 Toutes les tables sont prêtes !
Les données sont gérées automatiquement par Databricks


In [0]:
print("VÉRIFICATION DES TABLES")
print("=" * 70)

tables = spark.sql("SHOW TABLES IN formation").collect()

for table in tables:
    table_name = table.tableName
    count = spark.sql(f"SELECT COUNT(*) as cnt FROM formation.{table_name}").collect()[0]['cnt']
    print(f"{table_name:30} : {count} lignes")

print("\nTout est prêt pour les requêtes SQL !")

VÉRIFICATION DES TABLES
ventes_bronze                  : 8 lignes
ventes_gold_produit            : 4 lignes
ventes_gold_ville              : 3 lignes
ventes_silver                  : 6 lignes

Tout est prêt pour les requêtes SQL !


In [0]:
%sql
SELECT 
    id_commande,
    date_commande,
    client,
    produit,
    montant,
    ville,
    statut,
    categorie_prix
FROM formation.ventes_silver
ORDER BY montant DESC

id_commande,date_commande,client,produit,montant,ville,statut,categorie_prix
1,2025-10-01T10:30:00.000Z,Alice,Laptop,1200.0,Paris,Confirmée,Premium
5,2025-10-03T16:00:00.000Z,Emma,Laptop,1200.0,Lyon,Confirmée,Premium
4,2025-10-02T14:45:00.000Z,David,Écran,350.0,Marseille,Confirmée,Standard
3,2025-10-02T09:20:00.000Z,Claire,Clavier,75.0,Paris,Annulée,Standard
2,2025-10-01T11:15:00.000Z,Bob,Souris,25.5,Lyon,Confirmée,Économique
6,2025-10-03T17:30:00.000Z,Alice,Souris,25.5,Paris,Confirmée,Économique


In [0]:
%sql
-- Classement des villes
SELECT 
    ville,
    chiffre_affaires,
    nombre_commandes,
    ROUND(panier_moyen, 2) as panier_moyen
FROM formation.ventes_gold_ville
ORDER BY chiffre_affaires DESC

ville,chiffre_affaires,nombre_commandes,panier_moyen
Paris,1300.5,3,433.5
Lyon,1225.5,2,612.75
Marseille,350.0,1,350.0


In [0]:
%sql
-- Produits les plus vendus
SELECT 
    produit,
    quantite_vendue,
    ROUND(revenu_total, 2) as revenu_total,
    ROUND(prix_moyen, 2) as prix_moyen
FROM formation.ventes_gold_produit
ORDER BY quantite_vendue DESC

produit,quantite_vendue,revenu_total,prix_moyen
Souris,2,51.0,25.5
Laptop,2,2400.0,1200.0
Écran,1,350.0,350.0
Clavier,1,75.0,75.0


In [0]:
%sql
-- Produits les plus vendus
SELECT 
    produit,
    quantite_vendue,
    ROUND(revenu_total, 2) as revenu_total,
    ROUND(prix_moyen, 2) as prix_moyen
FROM formation.ventes_gold_produit
ORDER BY quantite_vendue DESC

produit,quantite_vendue,revenu_total,prix_moyen
Souris,2,51.0,25.5
Laptop,2,2400.0,1200.0
Écran,1,350.0,350.0
Clavier,1,75.0,75.0


In [0]:
%sql
-- Quel client a dépensé le plus ?
SELECT 
    client,
    ville,
    COUNT(*) as nombre_achats,
    SUM(montant) as total_depense,
    ROUND(AVG(montant), 2) as panier_moyen
FROM formation.ventes_silver
GROUP BY client, ville
ORDER BY total_depense DESC

client,ville,nombre_achats,total_depense,panier_moyen
Alice,Paris,2,1225.5,612.75
Emma,Lyon,1,1200.0,1200.0
David,Marseille,1,350.0,350.0
Claire,Paris,1,75.0,75.0
Bob,Lyon,1,25.5,25.5
